# 도구 (Tools)

LangChain에서의 **Tool(도구)** 는 언어 모델(LLM)이 외부 세계와 상호작용할 수 있도록 연결해주는 인터페이스입니다.
LLM이 단순히 텍스트만 생성하는 것을 넘어 실제적인 작업을 수행할 수 있게 만드는 기능의 단위입니다.  

**핵심역할**
- 기능확장 : 검색 엔진 사용, 계산기, 데이터베이스 조회, API 호출 등 LLM 자체의 능력을 넘어서는 작업 가능

### Tool의 핵심 구조
**name (이름)**

- Tool을 식별하는 고유한 이름입니다.
- LLM이 사용할 Tool을 선택할 때 이 이름을 사용합니다.
예: google-search, calculator, database-lookup

**description (설명)**

- 가장 중요한 요소로, 이 Tool이 무엇을 하는지, 언제 사용해야 하는지를 자연어로 상세하게 설명합니다.
- LLM은 이 설명을 보고 사용자의 질문 의도에 가장 적합한 Tool을 판단합니다. 
- 설명이 명확하고 구체적일수록 LLM이 Tool을 더 정확하게 선택할 수 있습니다.
예: "최신 정보를 검색해야 할 때 사용합니다. 구글 검색 엔진을 통해 웹에서 실시간 정보를 가져옵니다."

**args_schema (입력 스키마)**

- Tool을 실행하는 데 필요한 입력값(인자)의 종류와 형식을 정의합니다.
- Pydantic 모델을 사용하여 정의하며, 이를 통해 LLM은 어떤 정보를 어떤 형태로 Tool에 전달해야 하는지 알 수 있습니다.
예를 들어, 검색 Tool이라면 query라는 이름의 문자열(string) 타입 입력값이 필요하다고 정의할 수 있습니다.



### 실제 Agent 동작 방식
이 세 가지 요소를 바탕으로 LLM은 다음과 같이 작동합니다.
1. Tool 선택:  
- 사용자의 질문이 들어오면, LLM은 자신이 사용할 수 있는 모든 Tool의 **description**을 살펴보고 가장 적절한 Tool의 **name**을 선택합니다.
2. 입력값 추출:  
- 선택한 Tool의 **args_schema**를 보고, 사용자의 질문에서 필요한 입력값을 추출하여 형식에 맞게 정리합니다.
3. Tool 호출 요청:  
- 최종적으로 **{"tool": "선택한_tool_이름", "tool_input": {"인자_이름": "추출한_값"}}**과 같은 형식의 JSON 객체를 생성하여 반환합니다

## 사용자 정의 도구(Custom Tool)

커스텀 도구란, 사용자가 특정 목적을 위해 직접 정의한 기능을 LangChain 에이전트가 활용할 수 있도록 만든 도구를 의미합니다.
**내가 만든 파이썬 함수를 LLM(거대 언어 모델)이 이해하고 사용할 수 있는 형태로 포장가능**

### @tool 데코레이터
- 커스텀 도구를 만드는 가장 일반적이고 쉬운 방법은 @tool 데코레이터(decorator)를 사용하는 것입니다.

**사용 방법**
1. langchain_core.tools 라이브러리에서 tool을 가져옵니다.
2. 도구로 만들고 싶은 함수 위에 @tool을 붙여줍니다.
3. 함수의 docstring(독스트링, 설명 부분)에 도구의 설명을 명확하게 작성합니다.  
(이 설명은 LLM이 언제 이 도구를 사용해야 할지 판단하는 매우 중요한 기준이 됩니다.)

In [ ]:
import os
from dotenv import load_dotenv

# .env 파일에서 환경변수 로드
load_dotenv()

# OpenAI API 환경변수 값 확인
openai_api_key = os.getenv('OPENAI_API_KEY')
print(f"OPENAI_API_KEY가 설정되어 있나요?: {openai_api_key[:10]}...")

In [ ]:
from langchain.tools import tool

# 데코레이터를 통해 함수를 도구로 변환
@tool
def add_numbers(a: int, b: int) -> int: # input 정의
    """두 숫자를 더합니다."""

    return a + b # output 정의

@tool
def multiply_numbers(a: int, b: int) -> int:
    """두 숫자를 곱합니다."""

    return a * b

In [ ]:
add_numbers.invoke({"a":10, "b":20})

In [ ]:
multiply_numbers.invoke({"a":10, "b":20})

**사용 예시**
- 사내 데이터베이스 조회: 우리 회사 고객 정보를 조회하는 함수를 만들어 에이전트에게 제공
- 특정 API 연동: 환율 정보를 실시간으로 가져오는 API를 호출하는 도구 제작
- 복잡한 연산 수행: 특정 비즈니스 로직에 따른 복잡한 계산을 수행하는 함수를 도구화

## 랭체인에서 제공하는 자체 built-in-tools

LangChain은 다양한 작업을 위해 즉시 사용할 수 있는 여러 자체 도구(built-in tools)를 제공합니다.

- 정보검색 : Tavily Search, DuckDuckgoSearch, ..   
- 코드실행 : REPL Tool, ..   
- API연동 : Requests Tools, OpenWeatherMap, ..  
- Database : SQLDatabase Toolkit, ..  


**참고**
- [LangChain Tools/Toolkits](https://python.langchain.com/docs/integrations/tools/)

### Tavily
Tavily는 일반 사용자가 아닌 AI 에이전트 전용으로 설계된 검색 엔진입니다.  
AI가 웹상의 최신 정보나 전문 지식을 실시간으로 찾아 활용할 수 있도록 돕습니다

**API 키 발급 주소**
- https://app.tavily.com/

발급한 API 키를 환경변수에 설정합니다.

`.env` 파일에 아래와 같이 설정합니다.

```
TAVILY_API_KEY=tvly-[API KEY 입력...]
```


### TavilySearchResults

**설명**
- Tavily 검색 API를 쿼리하고 JSON 형식의 결과를 반환합니다.
- 포괄적이고 정확하며 신뢰할 수 있는 결과에 최적화된 검색 엔진입니다.
- 현재 이벤트에 대한 질문에 답변할 때 유용합니다.

**주요 매개변수**
- max_results: 검색 결과 수 지정 (기본값: 5)
- search_depth: 검색 깊이 조절 ("basic" 또는 "advanced")
- include_answer : 검색결과에 Tavily의 요약정보를 포함할지 결정정 (기본값 True)
- include_images : 검색결과에 이미지를 포함할지 결정
- include_row_content : 원본텍스트결과를 전체에 포함할지 결정 (True로하면 매우길수있음) (기본값 : False)
- include_domains: 특정 도메인으로 검색 범위 제한
- exclude_domains: 특정 도메인을 검색에서 제외

**반환 값**
- 검색 결과를 포함하는 JSON 형식의 문자열(url, content)

In [ ]:
%pip install langchain_community

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults

# 도구 생성
tool = TavilySearchResults(
    max_results=3, #관련성 높은 3개만 참조
    include_answer=False, # 요약정보 미포함
    include_raw_content=True, # 원본정보 포함
    include_images=True, # 이미지 포함
    search_depth="advanced", # or "basic" # 심층탐색
    include_domains=["naver.com"], #naver.com에서만 찾기
    exclude_domains = []
)

In [ ]:
# 도구 실행
tool.invoke({"query": "AI Agent에 대해 알려주세요"})

### Python REPL 도구

 LangChain 에이전트에게 파이썬 코드를 실시간으로 실행할 수 있는 능력을 부여하는 도구
- PythonREPLTool

**설명**
- Python 셸 환경을 제공합니다.
- 유효한 Python 명령어를 입력으로 받아 실행합니다.
- 결과를 보려면 print(...) 함수를 사용해야 합니다.

**주요 특징**
- 높은 유연성:  
전용 도구가 없는 새로운 유형의 요청이나 복잡한 문제에 직면했을 때, 즉석에서 코드를 생성하여 대응할 수 있습니다.  
- 복잡한 연산 및 로직 처리:  
단순 계산기를 넘어선 다단계 계산, 데이터 정렬, 조건부 필터링, 반복문 등 복잡한 프로그래밍 로직을 수행할 수 있습니다.
- 라이브러리 활용:  
실행 환경에 `NumPy`, `Pandas`와 같은 데이터 과학 라이브러리가 설치되어 있다면, 에이전트가 이를 `import`하여 분석작업 가능

**사용 방법**
- PythonREPLTool 인스턴스 생성
- run 또는 arun, invoke 메서드를 사용하여 Python 코드 실행

In [1]:
%pip install langchain_experimental

Note: you may need to restart the kernel to use updated packages.


In [10]:
from langchain_experimental.tools import PythonREPLTool

# 파이썬 코드를 실행하는 도구를 생성합니다.
python_tool = PythonREPLTool()

파이썬 코드를 실행하고 결과를 반환합니다.

In [11]:
print(python_tool.invoke("print(10+25+8+45+17)"))

Python REPL can execute arbitrary code. Use with caution.


105



아래는 LLM 에게 파이썬 코드를 작성하도록 요청하고 결과를 반환하는 예제입니다.

**흐름 정리**
1. LLM 모델에게 특정 작업을 수행하는 Python 코드를 작성하도록 요청합니다.
2. 작성된 코드를 실행하여 결과를 얻습니다.
3. 결과를 출력합니다.


In [12]:
import os
from langchain.chat_models import init_chat_model
from langchain.agents import AgentExecutor, create_react_agent
from langchain_experimental.tools import PythonREPLTool
from langchain.prompts import PromptTemplate


# 1. 도구 정의 (Python 코드 실행 도구)
tools = [PythonREPLTool()]

# 2. LLM 정의
llm = init_chat_model(model="gpt-4o", temperature=0)

# 3. 프롬프트 직접 만들기 (React 프롬프트)
template = """
Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}
"""

# 위 템플릿으로 프롬프트 객체 생성
prompt = PromptTemplate.from_template(template)

# 4. 에이전트 및 실행기 생성
agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# 5. 에이전트 실행
response = agent_executor.invoke({
    "input": "12에 34를 곱하고 56을 더하면 얼마야?"
})

# 6. 최종 결과 출력
print("[최종 답변]:")
print(response["output"])



> Entering new AgentExecutor chain...
To solve the problem, I need to multiply 12 by 34 and then add 56 to the result. 

Action: Python_REPL
Action Input: 
```python
result = 12 * 34 + 56
print(result)
```464
I now know the final answer.

Final Answer: 12에 34를 곱하고 56을 더하면 464입니다.

> Finished chain.
[최종 답변]:
12에 34를 곱하고 56을 더하면 464입니다.
